In [ ]:
!pip install wandb timm torchvision torch
!pip install datasets huggingface_hub gradio


In [ ]:
!pip install -q wandb torch torchvision
import wandb

# Login to W&B (you'll get a link in the output)
wandb.login()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
!git clone https://github.com/rmccorm4/Tiny-Imagenet-200.git
!mv Tiny-Imagenet-200 tiny_imagenet


Cloning into 'Tiny-Imagenet-200'...
remote: Enumerating objects: 7966, done.
remote: Total 7966 (delta 0), reused 0 (delta 0), pack-reused 7966 (from 1)
Receiving objects: 100% (7966/7966), 5.88 MiB | 3.66 MiB/s, done.
Resolving deltas: 100% (2655/2655), done.


In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip -d tiny_imagenet/


--2025-11-02 16:45:12--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-11-02 16:45:12--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  71.5MB/s    in 3.3s    

2025-11-02 16:45:15 (71.5 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
!ls tiny_imagenet/tiny-imagenet-200 | head


test
train
val
wnids.txt
words.txt


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torchvision import models
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [ ]:
data_dir = "tiny_imagenet/tiny-imagenet-200"

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load train/val sets
train_dataset = torchvision.datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root=f"{data_dir}/val", transform=transform)

# 🪄 Reduce dataset to small sample for quick training
small_train = Subset(train_dataset, range(1000))
small_val = Subset(val_dataset, range(200))

train_loader = torch.utils.data.DataLoader(small_train, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(small_val, batch_size=64, shuffle=False)

print("Train size:", len(small_train))
print("Val size:", len(small_val))


Train size: 1000
Val size: 200


In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)  # Tiny ImageNet = 200 classes
model = model.to(device)


In [ ]:
wandb.init(project="tiny-imagenet-resnet", config={
    "epochs": 3,
    "batch_size": 64,
    "lr": 1e-3,
    "optimizer": "Adam",
})

config = wandb.config
optimizer = optim.Adam(model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(config.epochs):
    model.train()
    train_loss, correct = 0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (outputs.argmax(1) == y).sum().item()

    train_acc = correct / len(small_train)
    wandb.log({"train_loss": train_loss / len(train_loader), "train_acc": train_acc})

    # Validation
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            val_loss += criterion(outputs, y).item()
            val_correct += (outputs.argmax(1) == y).sum().item()

    val_acc = val_correct / len(small_val)
    wandb.log({"val_loss": val_loss / len(val_loader), "val_acc": val_acc})

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")

wandb.finish()


Epoch 1: Train Acc=0.867, Val Acc=0.610
Epoch 2: Train Acc=0.970, Val Acc=0.200
Epoch 3: Train Acc=0.984, Val Acc=0.595


train_acc,▁▇█
train_loss,█▁▁
val_acc,█▁█
val_loss,▇█▁
train_acc,0.984
train_loss,0.053
val_acc,0.595
val_loss,1.57557


In [ ]:
torch.save(model.state_dict(), "resnet_tinyimagenet.pth")


In [ ]:
!pip install huggingface_hub gradio --quiet
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import create_repo

create_repo(repo_id="NiranjanIITPKD/tinyimagenet-resnet", repo_type="model", exist_ok=True)

RepoUrl('https://huggingface.co/NiranjanIITPKD/tinyimagenet-resnet', endpoint='https://huggingface.co', repo_type='model', repo_id='NiranjanIITPKD/tinyimagenet-resnet')

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="resnet_tinyimagenet.pth",
    path_in_repo="resnet_tinyimagenet.pth",
    repo_id="NiranjanIITPKD/tinyimagenet-resnet",
    repo_type="model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  resnet_tinyimagenet.pth     :   1%|1         |  566kB / 45.2MB            

CommitInfo(commit_url='https://huggingface.co/NiranjanIITPKD/tinyimagenet-resnet/commit/9ca313c3781f96f66a35347beca86ba2253daa95', commit_message='Upload resnet_tinyimagenet.pth with huggingface_hub', commit_description='', oid='9ca313c3781f96f66a35347beca86ba2253daa95', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NiranjanIITPKD/tinyimagenet-resnet', endpoint='https://huggingface.co', repo_type='model', repo_id='NiranjanIITPKD/tinyimagenet-resnet'), pr_revision=None, pr_num=None)

In [1]:
import gradio as gr
from PIL import Image
import torchvision.transforms as T

def predict(image):
    model.eval()
    transform = T.Compose([T.Resize((64,64)), T.ToTensor()])
    img = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        preds = model(img)
    return f"Predicted class ID: {preds.argmax(1).item()}"

demo = gr.Interface(fn=predict, inputs="image", outputs="text",
                    title="Tiny ImageNet ResNet Classifier")
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://619c2b1dadb00fb30a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from torchvision.transforms import functional as TF
import random

def drift_transform(img):
    if random.random() < 0.5:
        img = TF.adjust_brightness(img, brightness_factor=0.3)
    else:
        noise = torch.randn_like(TF.to_tensor(img)) * 0.2
        img = TF.to_pil_image(torch.clamp(TF.to_tensor(img) + noise, 0, 1))
    return img

drifted_dataset = datasets.ImageFolder(
    data_dir + '/val',
    transform=transforms.Compose([
        transforms.Resize((64,64)),
        transforms.Lambda(drift_transform),
        transforms.ToTensor()
    ])
)

drifted_loader = DataLoader(drifted_dataset, batch_size=64, shuffle=False)


In [ ]:
wandb.init(project="tiny-imagenet-resnet", name="drifted_run")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in drifted_loader:
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(1)
        correct += (preds == y).sum().item()

drifted_acc = correct / len(drifted_dataset)
wandb.log({"drifted_accuracy": drifted_acc})

if drifted_acc < 0.3:  # threshold for alert
    wandb.alert(
        title=" Accuracy Drop Detected",
        text=f"Drifted accuracy = {drifted_acc:.2f}, significantly lower than baseline."
    )

wandb.finish()


wandb: Detected [huggingface_hub.inference, mcp] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


drifted_accuracy,▁
drifted_accuracy,0.4788


In [ ]:
!pip install -q gradio huggingface_hub


In [ ]:
from huggingface_hub import create_repo, upload_folder

# 1. Create a Gradio Space (if not exists)
create_repo("tinyimagenet-resnet-space", repo_type="space", exist_ok=True, space_sdk="gradio")

# 2. Create a simple app.py for deployment
app_code = """
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models

model = models.resnet18(num_classes=200)
model.load_state_dict(torch.load("resnet_tinyimagenet.pth", map_location=torch.device("cpu")))
model.eval()

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

def predict(img):
    img = transform(img).unsqueeze(0)
    with torch.no_grad():
        preds = model(img)
        pred_class = preds.argmax(dim=1).item()
    return {f"Class {pred_class}": float(preds.max())}

gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Tiny ImageNet Classifier"
).launch()
"""

# Write the app.py file
with open("app.py", "w") as f:
    f.write(app_code)

# 3. Upload the app and model file to the Space
upload_folder(
    repo_id="NiranjanIITPKD/tinyimagenet-resnet-space",
    folder_path=".",  # current directory
    repo_type="space"
)


It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`hf upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...t/resnet_tinyimagenet.pth:  18%|#8        | 8.35MB / 45.2MB            

  ...ample_data/mnist_test.csv: 100%|##########| 18.3MB / 18.3MB            

  ...ata/mnist_train_small.csv:  69%|######8   | 25.1MB / 36.5MB            

  ...ent/tiny-imagenet-200.zip:   6%|6         | 15.6MB /  248MB            

HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/spaces/NiranjanIITPKD/tinyimagenet-resnet-space/commit/main

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "NiranjanIITPKD/tinyimagenet-resnet-space"

# Upload only required files
api.upload_file(
    path_or_fileobj="app.py",
    path_in_repo="app.py",
    repo_id=repo_id,
    repo_type="space"
)

api.upload_file(
    path_or_fileobj="resnet_tinyimagenet.pth",
    path_in_repo="resnet_tinyimagenet.pth",
    repo_id=repo_id,
    repo_type="space"
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  resnet_tinyimagenet.pth     :  74%|#######4  | 33.5MB / 45.2MB            

CommitInfo(commit_url='https://huggingface.co/spaces/NiranjanIITPKD/tinyimagenet-resnet-space/commit/a968b5ec5c07eaa56827b6dc021b27bd04d83af7', commit_message='Upload resnet_tinyimagenet.pth with huggingface_hub', commit_description='', oid='a968b5ec5c07eaa56827b6dc021b27bd04d83af7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/NiranjanIITPKD/tinyimagenet-resnet-space', endpoint='https://huggingface.co', repo_type='space', repo_id='NiranjanIITPKD/tinyimagenet-resnet-space'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="/content/wandb/latest-run/files/requirements.txt",
    path_in_repo="requirements.txt",
    repo_id="NiranjanIITPKD/tinyimagenet-resnet-space",
    repo_type="space",
    commit_message="Fix build: use minimal requirements.txt"
)


CommitInfo(commit_url='https://huggingface.co/spaces/NiranjanIITPKD/tinyimagenet-resnet-space/commit/6f13b770b7db125ae60287bd144a80baa7e3e131', commit_message='Fix build: use minimal requirements.txt', commit_description='', oid='6f13b770b7db125ae60287bd144a80baa7e3e131', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/NiranjanIITPKD/tinyimagenet-resnet-space', endpoint='https://huggingface.co', repo_type='space', repo_id='NiranjanIITPKD/tinyimagenet-resnet-space'), pr_revision=None, pr_num=None)